# Workload Effect on Pitcher Injury

## Goal: 
Determine if we can find:
1) Maximize the predictability of the probability of a pitcher getting injured
2) A causal effect on pitcher injury (e.g., workload, less rest, etc.)

## Background:
For our (1) goal, we want to predict the probability that a pitcher will get injured as a result of workload, rest (or lack thereof), distance traveled, etc. Essentially, given some features regarding what the pitcher has recently done, we can determine the probability that he'll get injured, should he play the next game. In a sense, we're trying to come up with a solution for teams', such that, based on the probability that results from the features we mentioned, a manager or top decision-maker would make judgdment (e.g., our probability induces a certain threshold such that meeting that threshold induces the idea of resting that pitcher instead of letting him pitch and risk an injury). 

Now we understand that there is one glaring issue, and that is that there are many other confounding factors that induce an injury that might not necessarily *be* correlated with workload (e.g., the ball comes back to the pitcher and hits them and takes them out, the pitcher's form wasn't quite right and it caused an inury as a result of straining your body to come up with velo). To account for this, we want to focus on types of injuries that we are confident *are* correlated with workload, and we'll use scientific evidence to support this (e.g., torn UCL). The reason to avoid those other "freak" types of injuries is that they are a potential source of noise/bias as mentioned previously, some injuries are just not a result of workload, meaning it is entirely possible that our models could *learn* from these types of injuries and determine some kind of relationship with the probability of an injury, which we would not want. 

To approach this problem, we want to take a step back and view this problem from another lens. That being: Industrial Engineering. Essentially, we want to view pitchers as "machines," and as a result, we want to estimate the "failure" rate (injury rate) of these "machines" as a function of workload cycles, rest, and travel. Meaning, we want to quantitatively come up with policies for the pitcher that are analogous to process optimizations. As we want to identify when the injury risk of these pitchers accelerate, such that we can prevent this *potential* injury in order to maximize pitcher usage, and minimize any waste, cost, or downtime.

In future iterations, we'd be interested in learning about *true* causal effects in terms of pitcher injuries, and whether or not the factors we talked about (e.g., workload, rest, etc.) have that kind of effect.

## Methodology:


To approach this problem, we first want to understand if there *are* any type of relationships between these workload variables and injury probability. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pybaseball as pyb
import requests

pd.set_option('display.max_columns', None)

In [2]:
mlb_injuries = pd.read_csv('mlb_injuries.csv')
mlb_injuries

,rank,player,pos,team,il_type,injury,start_date,end_date,reason_raw,year
0,1,Justin Verlander,SP,HOU,10-Day IL,Arm,2020-07-27,2020-09-27,10-Day IL - Arm: 7/27/20-9/27/20,2020
1,2,Chris Sale,SP,BOS,60-Day IL,Elbow Tommy John,2020-07-24,2020-09-27,60-Day IL - Elbow Tommy John: 7/24/20-9/27/20,2020
2,3,Stephen Strasburg,SP,WSH,60-Day IL,Hand,2020-08-15,2020-09-27,60-Day IL - Hand: 8/15/20-9/27/20,2020
3,4,Jordan Zimmermann,SP,DET,60-Day IL,Arm,2020-07-23,2020-09-10,60-Day IL - Arm: 7/23/20-9/10/20,2020
4,5,Cole Hamels,SP,ATL,60-Day IL,Arm,2020-07-23,2020-09-16,60-Day IL - Arm: 7/23/20-9/16/20 60-Day IL - ...,2020
...,...,...,...,...,...,...,...,...,...,...
5162,628,Cade Horton,SP,CHC,15-Day IL,Ribs,2025-09-25,2025-09-28,15-Day IL - Ribs: 9/25/25-9/28/25,2025
5163,629,Everson Pereira,OF,TB,10-Day IL,Back,2025-09-26,2025-09-28,10-Day IL - Back: 9/26/25-9/28/25,2025
5164,630,Brett Baty,2B,NYM,10-Day IL,Oblique,2025-09-27,2025-09-28,10-Day IL - Oblique: 9/27/25-9/28/25,2025
5165,631,Nick Frasso,SP,LAD,60-Day IL,Undisclosed,2025-09-27,2025-09-28,60-Day IL - Undisclosed: 9/27/25-9/28/25,2025


In [7]:
def get_name_birth(id: int):
    """
    A get function to find the birth year and full name of each player based on their mlb id using MLB's Stats API

    Note: you have to be online to run this
    """
    url = f"https://statsapi.mlb.com/api/v1/people/{id}"
    r = requests.get(url).json()
    return [r['people'][0]['birthDate'].split('-')[0], r['people'][0]['fullName'], r['people'][0]['primaryPosition']['abbreviation']]

In [53]:
def load_year_data(year: int):
    """
    Creates the relevant pitcher data for each given year
    Should take about 7-8 min to run
    """
    df = pyb.statcast(f'{year}-03-01', f'{year}-11-30').copy() ## chooses the specific year
    ## cleans up the pitch by pitch data for that season
    df = df[df['game_type'] == 'R'].sort_values(by=['game_date', 'game_pk', 'inning', 'at_bat_number'], ascending=True).reset_index(drop=True).copy()
    df['game_date'] = pd.to_datetime(df['game_date'])
    ## adds the pitch group to simplify pitch usage
    fastball_pitches = ['FF', 'FC', 'SI']
    breaking_pitches = ['CU', 'KC', 'SC', 'SL', 'SV', 'ST']
    offspeed_pitches = ['CH', 'FO', 'FS']
    df['pitch_group'] = df['pitch_type'].apply(lambda x: 'Fastball' if x in fastball_pitches 
                                               else 'Breakingball' if x in breaking_pitches 
                                               else 'Offspeed' if x in offspeed_pitches 
                                               else 'Other')
    ## finds all of the appearances each pitcher made in that season as well as the number of pitches thrown
    df_pitcher_games = df.groupby(['player_name', 'pitcher', 'game_pk', 'game_date'], as_index=False).agg(pitches_thrown=('pitcher', 'count')).copy()
    df_pitcher_games = df_pitcher_games.sort_values(['player_name', 'game_date']).copy()

    ## creates the indicator of what the current season is
    df_pitcher_games['season'] = df_pitcher_games['game_date'].dt.year
    ## finds the previous appearence made for each current date 
    df_pitcher_games['last_start_date'] = (df_pitcher_games.groupby('pitcher')['game_date'].shift(1))
    ## finds the number of pitches that were thrown in the previous appearance made
    df_pitcher_games['pitches_last_start'] = (df_pitcher_games.groupby('pitcher')['pitches_thrown'].shift(1))
    ## the number of rest days the pitcher had before their current appearence
    df_pitcher_games['days_since_last_start'] = ((df_pitcher_games['game_date'] - df_pitcher_games['last_start_date']).dt.days) - 1
    ## a counter for the number appeareance made during the season
    df_pitcher_games['number_start'] = (df_pitcher_games.groupby('pitcher').cumcount() + 1)
    ## a flag for if that appearence made was their first of the season
    df_pitcher_games['first_start'] = (df_pitcher_games['number_start'] == 1).astype(int)

    print('Starting the birthday process')
    ## finds who the real pitchers are in the dataset, as well as their birth year to find age -> takes about 3 min for one season's worth of pitchers
    pitcher_list = df_pitcher_games.groupby(['player_name', 'pitcher'])['number_start'].count().reset_index().copy()
    births = {i: get_name_birth(i) for i in pitcher_list['pitcher'].unique().tolist()}
    names_and_bdays = pd.DataFrame.from_dict(births, orient='index', columns=['birth_year', 'full_name', 'primary_pos']).reset_index(names='id')
    ## keeps it to only pitchers and excludes position players
    names_and_bdays = names_and_bdays[names_and_bdays['primary_pos'].isin(['P', 'TWP'])].reset_index(drop=True).copy() 
    names_and_bdays = names_and_bdays[['id', 'birth_year']].copy()
    print('Finished the birthday process')

    ## only includes the players that are actually pitchers + adds their birth year
    df_pitcher_games = df_pitcher_games[df_pitcher_games['pitcher'].isin(names_and_bdays['id'].unique())].reset_index(drop=True).copy()
    df_pitcher_games = df_pitcher_games.merge(names_and_bdays, how='left', left_on='pitcher', right_on='id').copy()

    ## creates the age column
    df_pitcher_games['age'] = df_pitcher_games['season'] - df_pitcher_games['birth_year'].astype(int)

    ## cleans up the data
    df_pitcher_games = df_pitcher_games[['season', 'player_name', 'pitcher', 'age', 'days_since_last_start', \
                                         'pitches_last_start', 'number_start', 'first_start', \
                                            'last_start_date', 'game_date']].reset_index(drop=True).copy()

    ## the pitch-data for each game for each pitcher
    pitches = (df
               .groupby(['player_name', 'pitcher', 'game_pk', 'game_date', 'pitch_group'], as_index=False)
               .agg(pitches_thrown=('pitch_group', 'count'), 
                    avg_release_speed=('release_speed', 'mean'),
                    avg_spin = ('release_spin_rate', 'mean')
                    )
               ).copy()
    pitches = pitches[pitches['pitch_group'] != 'Other'].reset_index(drop=True).copy()
    ## pivots the table to include columns for each pitch type
    df_wide = (pitches
               .pivot_table(index=['player_name', 'pitcher', 'game_pk', 'game_date'], columns='pitch_group', values=['pitches_thrown', 'avg_release_speed', 'avg_spin'])
               )
    df_wide.columns = [f"{pitch}_{metric}" for metric, pitch in df_wide.columns]
    df_wide = df_wide.reset_index().copy()
    ## normalizes the thrown pitches to be rates
    ## makes sure that the missing pitches are counted as 0 because they technically didn't throw that pitch
    df_wide['Breakingball_pitches_thrown'] = df_wide['Breakingball_pitches_thrown'].fillna(0)
    df_wide['Fastball_pitches_thrown'] = df_wide['Fastball_pitches_thrown'].fillna(0)
    df_wide['Offspeed_pitches_thrown'] = df_wide['Offspeed_pitches_thrown'].fillna(0)
    total_pitches_thrown = df_wide['Breakingball_pitches_thrown'] + df_wide['Fastball_pitches_thrown'] + df_wide['Offspeed_pitches_thrown']
    df_wide['Breakingball_pitches_thrown'] = df_wide['Breakingball_pitches_thrown'] / total_pitches_thrown
    df_wide['Fastball_pitches_thrown'] = df_wide['Fastball_pitches_thrown'] / total_pitches_thrown
    df_wide['Offspeed_pitches_thrown'] = df_wide['Offspeed_pitches_thrown'] / total_pitches_thrown
    df_wide = df_wide[['pitcher', 'game_date', 'Fastball_pitches_thrown', 'Fastball_avg_release_speed', 'Fastball_avg_spin',\
                       'Breakingball_pitches_thrown', 'Breakingball_avg_release_speed', 'Breakingball_avg_spin',\
                        'Offspeed_pitches_thrown', 'Offspeed_avg_release_speed', 'Offspeed_avg_spin'
                        ]]
    df_wide = df_wide.rename(columns={'Fastball_pitches_thrown': 'FB_usage', 'Fastball_avg_release_speed': 'FB_velo', 'Fastball_avg_spin': 'FB_spin',
                                      'Breakingball_pitches_thrown': 'BB_usage', 'Breakingball_avg_release_speed': 'BB_velo', 'Breakingball_avg_spin': 'BB_spin',
                                      'Offspeed_pitches_thrown': 'OS_usage', 'Offspeed_avg_release_speed': 'OS_velo', 'Offspeed_avg_spin': 'OS_spin'
                                      })

    ## adds the pitch level data to each game
    df_pitcher_games = df_pitcher_games.merge(df_wide, how='left', on=['pitcher', 'game_date']).copy()
    pitches_to_shift = ['FB_usage', 'FB_velo', 'FB_spin', 'BB_usage', 'BB_velo', 'BB_spin', 'OS_usage', 'OS_velo', 'OS_spin']
    df_pitcher_games[pitches_to_shift] = (df_pitcher_games.groupby('pitcher')[pitches_to_shift].shift(1))

    ## changes the format of the name to be normalized
    df_pitcher_games['player_name'] = df_pitcher_games['player_name'].apply(lambda x: x.split(', ')[1].strip() + ' ' + x.split(', ')[0].strip()).str.lower()

    return df_pitcher_games

In [ ]:
mlb_data = pd.concat([load_year_data(i) for i in range(2020, 2025)], ignore_index=True)

This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


 24%|██▎       | 23/97 [00:20<00:48,  1.54it/s]

In [ ]:
pitcher_injuries = mlb_injuries[mlb_injuries['pos'].isin(['SP', 'P', 'RP'])].sort_values(by='start_date', ascending=False).reset_index(drop=True).copy()

## fixing the missing data values for start_date and end_date
pitcher_injuries['start_date'] = pitcher_injuries.apply(lambda x: pd.to_datetime(
    '20' + x['reason_raw'].split(':')[1].split('-')[0].rsplit('/', 2)[2] 
    + '-' 
    + ('0' + x['reason_raw'].split(':')[1].split('-')[0].rsplit('/', 2)[0].strip() 
       if int(x['reason_raw'].split(':')[1].split('-')[0].rsplit('/', 2)[0].strip()) < 10 
       else x['reason_raw'].split(':')[1].split('-')[0].rsplit('/', 2)[0].strip()
    ) 
    + '-' 
    + ('0' + x['reason_raw'].split(':')[1].split('-')[0].rsplit('/', 2)[1].strip() 
       if int(x['reason_raw'].split(':')[1].split('-')[0].rsplit('/', 2)[1].strip()) < 10 
       else x['reason_raw'].split(':')[1].split('-')[0].rsplit('/', 2)[1].strip()
    )).date()
    if pd.isna(x['start_date']) else x['start_date'], axis=1)
pitcher_injuries['end_date'] = pitcher_injuries['end_date'].fillna('TBD')

pitcher_injuries = pitcher_injuries[['player', 'il_type', 'injury', 'start_date', 'year']].copy()
pitcher_injuries['player'] = pitcher_injuries['player'].str.lower()
pitcher_injuries

,player,il_type,injury,start_date,year
0,nick frasso,60-Day IL,Undisclosed,2025-09-27,2025
1,chris martin,15-Day IL,Shoulder,2025-09-26,2025
2,cade horton,15-Day IL,Ribs,2025-09-25,2025
3,josé berríos,15-Day IL,Elbow,2025-09-25,2025
4,luis garcia,15-Day IL,Thigh,2025-09-25,2025
...,...,...,...,...,...
3006,daniel tillo,10-Day IL,Elbow,2020-07-23,2020
3007,wil crowe,60-Day IL,Shoulder,2023-04-23,2023
3008,jean carlos mejia,60-Day IL,Shoulder,2023-08-15,2023
3009,austin pruitt,60-Day IL,Knee,2024-04-21,2024


In [ ]:
mlb_data = mlb_data[mlb_data['first_start'] != 1]

,season,player_name,pitcher,age,days_since_last_start,pitches_last_start,number_start,first_start,last_start_date,game_date,FB_usage,FB_velo,FB_spin,BB_usage,BB_velo,BB_spin,OS_usage,OS_velo,OS_spin
1,2020,albert abreu,656061,25,25.0,41,2,0,2020-08-08,2020-09-03,0.392157,96.205,1994.8,0.215686,84.636364,2243.0,0.196078,86.68,1934.0
3,2020,bryan abreu,650556,23,2.0,31,2,0,2020-07-26,2020-07-29,0.205128,93.275,2320.875,0.589744,84.291304,2583.782609,NaN,<NA>,<NA>
4,2020,bryan abreu,650556,23,1.0,5,3,0,2020-07-29,2020-07-31,0.166667,93.6,2292.0,0.666667,84.875,2543.5,NaN,<NA>,<NA>
5,2020,bryan abreu,650556,23,3.0,16,4,0,2020-07-31,2020-08-04,0.272727,92.583333,2297.5,0.454545,84.01,2618.3,NaN,<NA>,<NA>
7,2020,jason adam,592094,29,2.0,25,2,0,2020-08-16,2020-08-19,0.450000,95.15,2712.888889,0.100000,81.675,2855.0,0.075000,89.666667,2045.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91341,2024,brett de geus,676969,27,0.0,20,11,0,2024-08-23,2024-08-24,0.411765,95.407143,2182.214286,0.176471,86.066667,2166.333333,NaN,<NA>,<NA>
91342,2024,brett de geus,676969,27,33.0,8,12,0,2024-08-24,2024-09-27,NaN,91.885714,2138.0,NaN,<NA>,<NA>,NaN,87.1,1372.0
91343,2024,brett de geus,676969,27,1.0,27,13,0,2024-09-27,2024-09-29,0.425532,94.365,2207.15,0.148936,83.771429,2465.571429,NaN,<NA>,<NA>
91345,2024,jacob degrom,594798,36,6.0,61,2,0,2024-09-13,2024-09-20,0.309524,97.169231,2522.230769,0.380952,89.365625,2669.15625,0.035714,90.5,1608.333333
